## **Transportation**


## **Integrated Dataset of Brazilian Flights**



### ** *Datasets* Preparation for Analysis**

**Abstract** This notebook provides information about the methods and processes used to improve data in the regular flights (VRA) dataset provided by ANAC (Agência Nacional de Aviação Civil) and introduces ASOS (Automated Surface Observing Systems) dataset provided by the IOWA University (USA) with weather sensor data installed at airports around the world.

**1. Introduction**

The VRA dataset contains departure and arrival data for brazilian domestic flights with improvements in the loading process were carried out in order to:

1. Improve data quality;
2. Make this dataset easier to use; and
3. Contemplate the period from 2000 to 2019 in a single dataset;

Additionally, a loading process was also created with climate data at airports in Brazil,
more complete and consistent. And this new data set is called ASOS (Automated Surface Observing Systems).

**1.1. VRA**

The data in this dataset are registered by the airlines and consolidated by ANAC (Agência
Nacional de Aviação Civil) in the database VRA (Regular Active Flights). Through the URL http://www.anac.gov.br/assuntos/dados-e-estatisticas/historico-de-voos it is possible to consult and
obtain data for the period from Jan 1st, 2000 to Dec 31st, 2019.

**1.2. ASOS**

ASOS (Automated Surface Observing Systems) is a program that involves several agencies
American government agencies, created with the objective of forming an official network of
meteorological information to support primarily aviation entities,
but also those focused on meteorological, climatological and hydrological research.

The Department of Agronomy at Iowa State University, in the United States, compiles daily
not only information from the US ASOS system, but also from various entities linked to
civil and military aviation from all over the planet.

This university makes this data freely available for download through the website
https://mesonet.agron.iastate.edu/request/download.phtml

The data treated and made available through this work were obtained at that address.

**1.3. Airports**

It was downloaded from http://openflights.org an *.dat* file with 7.798 airports around the world. Based on this file, it was created an airport dataset to be used in this work. OpenFlights.org is a tool that lets you map your flights around the world, search and filter them in all sorts of interesting ways, calculate statistics automatically, and share your flights and trips with friends and the entire world (if you wish). It's also the name of the open-source project to build the tool. OpenFlights is free offered by Contentshare. 

**2. Development**

The activities were developed individually for each dataset and are presented
in this report in the same way.

**2.1. VRA**

The VRA dataset used in this work consists of 20 .CSV (comma-separated values) files 
containing flight departure and arrival data, in the years 2000 to
2019.

Each .CSV file corresponds to one year (except for the months of June, July 2014 and March 2018 when ANAC did not collect the data). The following describes the problems encountered in analyzing the dataset and the solutions applied to
each one.

https://www.anac.gov.br/acesso-a-informacao/dados-abertos/areas-de-atuacao/voos-e-operacoes-aereas/voo-regular-ativo-vra/62-voo-regular-ativo-vra

The VRA metadados is found in the link above.

**2.1.1. Files Formatting**

The .CSV files made available by ANAC, each correspond to one month of data. So,
it was necessary to create a .BAT (Batch File) script with commands to execute the concatention of the twelve
files corresponding to the 12 months of each year, in a single .CSV file for each year.

For example: COPY *.CSV VRAYYYY.CSV where YYYY corresponds to an one year.

**2.1.2. Dataset generation**

The readVRA.R script below was created for reading the CSV files and creating the corresponding dataframe. The result is the 5.9 GB vra_2000_2019.RData file that, when loaded into the environment, provides the vra dataset with 3.7 GB and 20,206,673 observations or tuples with 22 characteristics or variables.

In [ ]:
print("Loading packages...")

# load doParallel
library(doParallel)
library(magrittr)
library(dplyr)
library(readxl)
library(sqldf)
library(psych)
library(ggplot2)
library(ggpubr)

print("Packages successfully loaded!")
print("workers configuring...")

# register with 2 cores for example
registerDoParallel(cores=2)
print("Workers successfully configured!")
print("Work directory configuring...")

# set the work directory for home
#setwd("~")
setwd("/home/cteixeira/DATAPAPER")
print("Work directory successfully configured!")
print("Creating list with dataframes...")

# define the initial name of the dataset
fileName<-"VRA"

# create a list of dataframes
vraList <- foreach(i=0:19) %dopar% {
  if(i<10) {
    read.csv(paste0(fileName,"200",
                    paste0(as.character(i),".CSV")),
             header=TRUE,stringsAsFactors=FALSE, sep=";")
  } else {
    read.csv(paste0(fileName,"20",
                    paste0(as.character(i),".CSV")),
             header=TRUE,stringsAsFactors=FALSE, sep=";")
  }
}

print("List with dataframes created successfully!")
print("Removing columns that will not be used...")

# Remove the columns not interested
for (i in 1:20) {
  vraList[[i]]$cd_di<-NULL
  vraList[[i]]$sg_equip<-NULL
  vraList[[i]]$nr_assentos<-NULL
  vraList[[i]]$sg_equip<-NULL
}

print("Columns that will not be used successfully removed!")

print("Binds all dataframes...")

# Binds all dataframes
vra<-do.call("rbind", vraList)

rm("vraList")
rm("fileName")
rm("i")

#-------------------- end of vra reading

#-------------------- begin preprocessing

#---- transformations

# convert to datetime objects
vra$dt_partida_prevista<-strptime(vra$dt_partida_prevista,"%d/%m/%Y %H:%M", tz="GMT")
vra$dt_partida_real<-strptime(vra$dt_partida_real,"%d/%m/%Y %H:%M", tz="GMT")

vra$dt_chegada_prevista<-strptime(vra$dt_chegada_prevista,"%d/%m/%Y %H:%M", tz="GMT")
vra$dt_chegada_real<-strptime(vra$dt_chegada_real,"%d/%m/%Y %H:%M", tz="GMT")

# Difference in Minutes for departures
vra$atraso_partida<-as.numeric(difftime(vra$dt_partida_real, vra$dt_partida_prevista, units = "mins"))

# Difference in Minutes for arrivals
vra$atraso_chegada<-as.numeric(difftime(vra$dt_chegada_real, vra$dt_chegada_prevista, units = "mins"))

# Difference in Minutes for expected duration
vra$duracao_esperada<-as.numeric(difftime(vra$dt_chegada_prevista, vra$dt_partida_prevista, units = "mins"))

# Difference in Minutes for real duration
vra$duracao_real<-as.numeric(difftime(vra$dt_chegada_real, vra$dt_partida_real, units = "mins"))

#--- create vra route column
#--- with the pair of airports of origin and destination respectively
vra$route <- paste0(vra$sg_origem, vra$sg_destino)

#--- create tp_origem and tp_destino column
vra$tp_origem=""
vra$tp_destino=""

#---- data cleaning

#--- Remove flights that flight_id not filled
vra<-vra %>% filter (vra$nr_voo!='')

#---- remove flights that are not in the dataset scope

#--- compute if origin and destination is a brazilian airport ('B') or other contry ('O')
#--- and fill tp_origem and tp_destino respectivelly
vra <- transform(vra, tp_origem= ifelse(grepl('^SB', sg_origem), "B", "O"))
vra <- transform(vra, tp_destino= ifelse(grepl('^SB', sg_destino), "B", "O"))

#--- if origin and destination are 'O' then tp_voo = 'R' means 'remove'
vra <- transform(vra, tp_voo= ifelse(tp_origem=="O" & tp_destino=="O", "R", "M"))

#--- remove flights where origin and destination is not a brazilian airport
vra <- vra %>% filter((tp_voo=="M"))

#--- remove tp_voo
vra$tp_voo<-NULL

#--- aerodromes started with 1, 2, SD, SI, SJ, SN, SS and SW are Brazilian aerodromes
vra$tp_origem = ifelse (grepl('^1', vra$sg_origem), "B", vra$tp_origem)
vra$tp_origem = ifelse (grepl('^2', vra$sg_origem), "B", vra$tp_origem)
vra$tp_origem = ifelse (grepl('^SD', vra$sg_origem), "B", vra$tp_origem)
vra$tp_origem = ifelse (grepl('^SI', vra$sg_origem), "B", vra$tp_origem)
vra$tp_origem = ifelse (grepl('^SJ', vra$sg_origem), "B", vra$tp_origem)
vra$tp_origem = ifelse (grepl('^SN', vra$sg_origem), "B", vra$tp_origem)
vra$tp_origem = ifelse (grepl('^SS', vra$sg_origem), "B", vra$tp_origem)
vra$tp_origem = ifelse (grepl('^SW', vra$sg_origem), "B", vra$tp_origem)

vra$tp_destino = ifelse (grepl('^1', vra$sg_destino), "B", vra$tp_destino)
vra$tp_destino = ifelse (grepl('^2', vra$sg_destino), "B", vra$tp_destino)
vra$tp_destino = ifelse (grepl('^SD', vra$sg_destino), "B", vra$tp_destino)
vra$tp_destino = ifelse (grepl('^SI', vra$sg_destino), "B", vra$tp_destino)
vra$tp_destino = ifelse (grepl('^SJ', vra$sg_destino), "B", vra$tp_destino)
vra$tp_destino = ifelse (grepl('^SN', vra$sg_destino), "B", vra$tp_destino)
vra$tp_destino = ifelse (grepl('^SS', vra$sg_destino), "B", vra$tp_destino)
vra$tp_destino = ifelse (grepl('^SW', vra$sg_destino), "B", vra$tp_destino)

# remove departure and arrival delays greater than 1440min(24h)
vra <- vra %>%
  filter (atraso_partida<1440)

vra <- vra %>%
  filter (atraso_chegada<1440)

#--- filter expected departures <= expected arrivals
vra <- vra %>% filter((dt_partida_prevista <= dt_chegada_prevista))

#--- filter real departures <= real arrivals
vra <- vra %>% filter((dt_partida_real <= dt_chegada_real))

#--- Remove flights that expected and real departure and arrival dates are not filled
vra<-vra %>% filter (!is.na(vra$dt_partida_prevista) |
                       !is.na(vra$dt_partida_real) |
                       !is.na(vra$dt_chegada_prevista) |
                       !is.na(vra$dt_chegada_real))

#--- Order vra by route
vra<-vra[order(vra$route),]

#--- Remove routes that have not been properly formed
vra<-vra %>% filter (nchar(vra$route)==8)
vra<-vra %>% filter (substr(vra$route,1,1)!='?')
vra<-vra %>% filter (substr(vra$route,1,4)!='0   ')

#--- control point
save(vra, file="vra_before_outliers_removal.rda")

#--- start outliers removal

#--- Order vra by route
vra<-vra[order(vra$route),]

#--- Create temporary columns for real duration of a flight with Outlier Thresholding
vra$dur_real_q1<-0
vra$dur_real_q3<-0
vra$dur_real_iqr<-0
vra$dur_real_th_low<-0
vra$dur_real_th_top<-0

vra$dur_esp_q1<-0
vra$dur_esp_q3<-0
vra$dur_esp_iqr<-0
vra$dur_esp_th_low<-0
vra$dur_esp_th_top<-0

# ----- Aldebaran ------
# architecture: x86_64 Intel(R) core(TM) i7-7820X CPU@ 3.60 GHz
# cpu op-modes: 32 or 64 bits
# cpu: 16 [0-15] detectCores()=16 cores
# threads per core: 2
# cores per socket: 8
# sockets: 1
library(doParallel)
registerDoParallel(cores=detectCores()-1)

#--- load functions
parallel_ComputeQ1 <- function(vra, s, var) {
  r <- foreach(s) %dopar% {
    quantile(vra[vra$route == s, var], c(0.25), na.rm=TRUE)
  }
  return(r)
}

parallel_ComputeQ3 <- function(vra, s, var) {
  r <- foreach(s) %dopar% {
    quantile(vra[vra$route == s, var], c(0.75), na.rm=TRUE)
  }
  return(r)
}

parallel_ComputeIQR <- function(vra, s, varq1, varq3) {
  r <- foreach(s) %dopar% {
    (vra[vra$route == s, varq3]-vra[vra$route == s, varq1])
  }
  return(r)
}

parallel_ComputeDurThLow <- function(vra, s, varq1, variqr) {
  r <- foreach(s) %dopar% {
    vra[vra$route == s, varq1] - 3 * vra[vra$route == s, variqr]
  }
  return(r)
}

parallel_ComputeDurThTop <- function(vra, s, varq3, variqr) {
  r <- foreach(s) %dopar% {
    vra[vra$route == s, varq3] + 3 * vra[vra$route == s, variqr]
  }
  return(r)
}

#--- measure the execution time
start_time<-Sys.time()
print(paste("Start time: ", start_time, sep=""))

print("Computing outliers thresholds for expected and real duration...")

#--- for each pair of airports (route)
#--- compute the outliers thresholds for expected and real duration variables
w<-0
l<-length(unique(vra$route))
for (s in unique(vra$route)) {
  print(print(paste("Route: ", s, sep="")))
  w<-w+1
  print(print(paste("% Work Done: ", paste((w/l)*100,"%",sep=""), sep="")))

  vra[vra$route == s, 'dur_esp_q1'] = parallel_ComputeQ1(vra, s, 'duracao_esperada')
  vra[vra$route == s, 'dur_esp_q3'] = parallel_ComputeQ3(vra, s, 'duracao_esperada')
  vra[vra$route == s, 'dur_esp_iqr'] = parallel_ComputeIQR(vra, s, 'dur_esp_q1', 'dur_esp_q3')
  vra[vra$route == s, 'dur_esp_th_low'] = parallel_ComputeDurThLow(vra, s, 'dur_esp_q1', 'dur_esp_iqr')
  vra[vra$route == s, 'dur_esp_th_top'] = parallel_ComputeDurThTop(vra, s, 'dur_esp_q3', 'dur_esp_iqr')

  vra[vra$route == s, 'dur_real_q1'] = parallel_ComputeQ1(vra, s, 'duracao_real')
  vra[vra$route == s, 'dur_real_q3'] = parallel_ComputeQ3(vra, s, 'duracao_real')
  vra[vra$route == s, 'dur_real_iqr'] = parallel_ComputeIQR(vra, s, 'dur_real_q1', 'dur_real_q3')
  vra[vra$route == s, 'dur_real_th_low'] = parallel_ComputeDurThLow(vra, s, 'dur_real_q1', 'dur_real_iqr')
  vra[vra$route == s, 'dur_real_th_top'] = parallel_ComputeDurThTop(vra, s, 'dur_real_q3', 'dur_real_iqr')  
}

end_time<-Sys.time()
print(paste("End time: ", end_time, sep=""))
print(paste("Execution time elapsed: ", end_time - start_time, sep=""))

#--- Unregister the parallel backend
env <- foreach:::.foreachGlobals
rm(list=ls(name=env), pos=env)
rm("env")

# remove flights with duracao_esperada and duracao_real are NA
vra <- transform(vra, remove= ifelse(is.na(vra$duracao_esperada)&is.na(vra$duracao_real),
                                     "Y", "N"))
vra <- vra %>% filter((remove=="N"))
vra$remove<-NULL

# remove flights with duracao_real less than low threshold
# and duracao_real greather than top threshold
vra <- transform(vra, remove= ifelse(is.na(vra$duracao_esperada),"N", "O"))
vra <- transform(vra, remove= ifelse(vra$remove=="O"&(vra$duracao_esperada<vra$dur_esp_th_low),"Y", "N"))
vra <- vra %>% filter((remove=="N"))
vra$remove<-NULL

vra <- transform(vra, remove= ifelse(is.na(vra$duracao_esperada),"N", "O"))
vra <- transform(vra, remove= ifelse(vra$remove=="O"&(vra$duracao_esperada>vra$dur_esp_th_top),"Y", "N"))
vra <- vra %>% filter((remove=="N"))
vra$remove<-NULL

vra <- transform(vra, remove= ifelse(is.na(vra$duracao_real),"N", "O"))
vra <- transform(vra, remove= ifelse(vra$remove=="O"&(vra$duracao_real<vra$dur_real_th_low),"Y", "N"))
vra <- vra %>% filter((remove=="N"))
vra$remove<-NULL

vra <- transform(vra, remove= ifelse(is.na(vra$duracao_real),"N", "O"))
vra <- transform(vra, remove= ifelse(vra$remove=="O"&(vra$duracao_real>vra$dur_real_th_top),"Y", "N"))
vra <- vra %>% filter((remove=="N"))
vra$remove<-NULL

#--- Remove temporary columns for real duration of a flight with Outlier Thresholding
vra$dur_real_q1<-NULL
vra$dur_real_q3<-NULL
vra$dur_real_iqr<-NULL
vra$dur_real_th_low<-NULL
vra$dur_real_th_top<-NULL

vra$dur_esp_q1<-NULL
vra$dur_esp_q3<-NULL
vra$dur_esp_iqr<-NULL
vra$dur_esp_th_low<-NULL
vra$dur_esp_th_top<-NULL

#--- end of outliers removal

#--- Compute and save flights per route
vra_routes_flights_2000_2019 <- vra %>%
  group_by(route) %>%
  dplyr::summarize(qtd = n())
vra_routes_flights_2000_2019<-vra_routes_flights_2000_2019[order(-vra_routes_flights_2000_2019$qtd),]
save(vra_routes_flights_2000_2019, file="vra_routes_flights_2000_2019.rda")
rm("vra_routes_flights_2000_2019")

#--- compute canceled flights per route
canceled_flights <- vra %>% filter(grepl('^X',vra$cd_justificativa))
save(canceled_flights, file="vra_canceled_flights_2000_2019.rda")
rm("canceled_flights")

#--- transform POSIXlt datetime fields to character
vra$dt_partida_prevista<-as.character(vra$dt_partida_prevista)
vra$dt_partida_real<-as.character(vra$dt_partida_real)
vra$dt_chegada_prevista<-as.character(vra$dt_chegada_prevista)
vra$dt_chegada_real<-as.character(vra$dt_chegada_real)

#--- create time fields from date fields
vra$hr_partida_prevista<-substr(vra$dt_partida_prevista,12,2)
vra$hr_partida_real<-substr(vra$dt_partida_real,12,2)
vra$hr_chegada_prevista<-substr(vra$dt_chegada_prevista,12,2)
vra$hr_chegada_real<-substr(vra$dt_chegada_real,12,2)

#--- reorder fields
vra = dplyr::select(vra, sg_empresa, nm_empresa, nr_voo, route, cd_tipo_linha,
                   sg_origem, tp_origem, nm_aerodromo_origem,
                   dt_partida_prevista, duracao_esperada, dt_partida_real, atraso_partida,
                   sg_destino, tp_destino, nm_aerodromo_destino,
                   dt_chegada_prevista, duracao_real, dt_chegada_real, atraso_chegada,
                   tp_situacao,
                   cd_justificativa, ds_justificativa)

print("Saving the VRA_2000_2019.RData dataset ...")

# Save the final dataset
save(vra, file="vra_2000_2019.rda")

print("VRA_2000_2019.RData Dataset successfully saved!")
print("Cleaning the work environment...")

rm("end_time")
rm("start_time")
rm("l")
rm("s")
rm("w")

# remove functions
rm("parallel_ComputeDurThLow")
rm("parallel_ComputeDurThTop")
rm("parallel_ComputeIQR")
rm("parallel_ComputeQ1")
rm("parallel_ComputeQ3")

# Remove the dataset
rm("vra")

# Clear the console
# CRTL+L to console and clear the screen
cat("\014")
print("Work environment Ready!")

[1] "Loading packages..."


Loading required package: foreach

Loading required package: iterators

Loading required package: parallel


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: gsubfn

Loading required package: proto

Warning message:
“no DISPLAY variable so Tk is not available”
Loading required package: RSQLite


Attaching package: ‘ggplot2’


The following objects are masked from ‘package:psych’:

    %+%, alpha




[1] "Packages successfully loaded!"
[1] "workers configuring..."
[1] "Workers successfully configured!"
[1] "Work directory configuring..."
[1] "Work directory successfully configured!"
[1] "Creating list with dataframes..."


**2.2. ASOS**

**2.2.1. Obtaining the data**

The data that makes up the ASOS dataset was downloaded from the Iowa State Department of Agronomy
University, through the website mentioned in the introduction to this report. The download was made from
as follows:

1. After accessing the IOWA University website, the first step is to select the network of stations
weather data to be obtained by clicking on the "Select Network" combobox,
selecting the option Brazil ASOS. Once selected, click on the "Switch to Network" button;

2. The website will then make available for selection, all the weather stations that are
found in the main aerodromes and airports in Brazil, as can be seen in the
map below.
<img src="ASOS BR.jpg">

3. The main aerodromes and airports in Brazil are ordered by the ICAO code (International Civil Aviation Organization), which is an international UN agency
United Nations) created to codify the main air navigation techniques and plan and
develop international air transport, ensuring its safety and development.

4. Once you have selected the BR_ASOS network, you can select all available stations in the "Avaiable Stations" combobox and add them to the "Selected Stations" combobox using the "Add All" button.

5. With all stations selected, the next step is to select the available data as presented in the "Select From Avaiable Data" combobox. In this work, we chose to select all available data by selecting the option "All Avaiable".

6. Next, select the desired time period. For this work, the complete period from 2000 to 2018 was considered. That said, in the year, month and day comboboxs for the "Start Date" field, the dates were selected, as shown in the Figure below.

<img src="DateRange.jpg">

7. Once the start and end dates are defined, you have the option of choosing the time zone. For the purposes of this work, the selection of the UTC (Universal Time Coordinated) option was maintained as a reference in the combobox "Timezone of Observation Times".

8. With all the functional requirements configured, the technical requirements are configured.

9. The first step in configuring the technical requirements is to define the download options, using the following comboboxs, as shown in the Figure below.

<img src="DowloadOptions.jpg">

10. To finish, keep the report's limit options and click on the "Get Data" button

<img src="GetData.jpg">

All 154 aerodromes in Brazil were included in the consultations.

As a result, a single 2.3GB text file (.TXT) was downloaded representing the complete years 2000 to 2018.

**2.2.2. Dataset generation**

The readAsos.R script below was created for reading the text file and creating the corresponding data frame. The result is the asosBR data set with 7,608,010 changes or tuples and 29 variables or characteristics.

**station:** three or four character site identifier<BR>
**valid:** timestamp of the observation<BR>
**tmpf:** Air Temperature in Fahrenheit, typically @ 2 meters<BR>
**dwpf:** Dew Point Temperature in Fahrenheit, typically @ 2 meters<BR>
**relh:** Relative Humidity in %<BR>
**drct:** Wind Direction in degrees from north<BR>
**sknt:** Wind Speed in knots<BR>
**p01i:** One hour precipitation for the period from the observation time to the time of the previous hourly precipitation reset. This varies slightly by site. Values are in inches. This value may or may not contain frozen precipitation melted by some device on the sensor or estimated by some other means. Unfortunately, we do not know of an authoritative database denoting which station has which sensor.<BR>
**alti:** Pressure altimeter in inches<BR>
**mslp:** Sea Level Pressure in millibar<BR>
**vsby:** Visibility in miles<BR>
**gust:** Wind Gust in knots<BR>
**skyc1:** Sky Level 1 Coverage<BR>
**skyc2:** Sky Level 2 Coverage<BR>
**skyc3:** Sky Level 3 Coverage<BR>
**skyc4:** Sky Level 4 Coverage<BR>
**skyl1:** Sky Level 1 Altitude in feet<BR>
**skyl2:** Sky Level 2 Altitude in feet<BR>
**skyl3:** Sky Level 3 Altitude in feet<BR>
**skyl4:** Sky Level 4 Altitude in feet<BR>
**wxcodes:** Present Weather Codes (space seperated)<BR>
**feel:** Apparent Temperature (Wind Chill or Heat Index) in Fahrenheit<BR>
**ice_accretion_1hr:** Ice Accretion over 1 Hour (inches)<BR>
**ice_accretion_3hr:** Ice Accretion over 3 Hours (inches)<BR>
**ice_accretion_6hr:** Ice Accretion over 6 Hours (inches)<BR>
**peak_wind_gust:** Peak Wind Gust (from PK WND METAR remark) (knots)<BR>
**peak_wind_drct:** Peak Wind Gust Direction (from PK WND METAR remark) (deg)<BR>
**peak_wind_time:** Peak Wind Gust Time (from PK WND METAR remark)<BR>
**metar:** unprocessed reported observation in METAR format

In [ ]:
#-----------------------------------------------------------------------------

#-- The R base function read.table() is a general function that
#-- can be used to read a file in table format.
#-- The data will be imported as a data frame.

#-- DO NOT RUN ... JUST TO ILUSTRATE ... WAS INCORPORATED INTO THE SCRIPT BELOW

asosBR <- as.data.frame(read.table("asosBR.txt", header = TRUE,
                                   sep = ",", dec = ".", skip = 0,
           stringsAsFactors=FALSE))

**3. Integration of VRA and ASOS datasets**

The vra_asosBR_Integration_v3.R script below was created to perform data integration between VRA and ASOS.

In [2]:
#--- Load libraries
print("Loading work packages...")

library("network")
library("plyr")
library("proto")
library("RSQLite")
library("gsubfn")
library("sqldf")
library("Matrix")
library("grid")
library("dplyr")
library("readxl")

print("Work packages loaded successfully!")

# set the work directory for home
setwd("/home/cteixeira/DATAPAPER")
print("Work directory successfully configured!")

#--- Load VRA
print("Loading vra_2000_2019.rda...")

load("vra_2000_2019.rda")

print("vra_2000_2019.rda uploaded successfully!")

# compute the original number of tuples
originalVRAtuples<-nrow(vra)

print(paste("Tuples qty. in original dataset:",originalVRAtuples))

#--- Load AIRPORTS
print("Loading airports.rda...")

load("airports.rda")

print("airports.rda uploaded successfully!")

#--- Load AIRLINES
print("Loading airlines.rda...")

load("airlines.rda")

print("airlines.rda uploaded successfully!")

#--- Load JUSTIFICATIONS
print("Loading justifications.rda...")

load("justifications.rda")

print("justifications.rda uploaded successfully!")

#--- Load ORIGINS
print("Loading origins.rda...")

load("origins.rda")

print("origins.rda uploaded successfully!")

#--- Load COUNTRIES
print("Loading countries.rda...")

load("countries.rda")

print("countries.rda uploaded successfully!")

#--- for the origins
vra <- vra[order(vra$dt_partida_real, vra$sg_origem),]

#--- merge VRA with origins
#--- in order to fill the missing airports names
df<-merge(vra, origins,
          by.x=c("sg_origem"),
          by.y=c("origin_icao"),
          all.x=TRUE,
          sort=TRUE)

#--- adjust the names 
df$tp_origem.x<-NULL
df$nm_aerodromo_origem.x<-NULL
names(df)[names(df) == "origin_type.y"] <- "tp_origem"
names(df)[names(df) == "origin_name.y"] <- "nm_aerodromo_origem"

#--- reorder fields
df = dplyr::select(df, sg_empresa, nm_empresa, nr_voo, route, cd_tipo_linha,
                   sg_origem, tp_origem, nm_aerodromo_origem,
                   dt_partida_prevista, duracao_esperada, dt_partida_real, atraso_partida,
                   sg_destino, tp_destino, nm_aerodromo_destino,
                   dt_chegada_prevista, dt_chegada_real, duracao_real, atraso_chegada, tp_situacao,
                   cd_justificativa, ds_justificativa)

#--- kill old dataframe
rm("vra")

#--- rename the new one
vra <- df

#--- kill temporary dataframe
rm("df")

#--- fill nm_aerodromo_origem not found
df<-merge(vra, airports,
          by.x=c("sg_origem"),
          by.y=c("airport_icao"),
          all.x=TRUE,
          sort=TRUE)

df$nm_aerodromo_origem <- ifelse(is.na(df$nm_aerodromo_origem),
                                 df$airport_name, df$nm_aerodromo_origem)

df$airport_name<-NULL

#--- reorder fields
df = dplyr::select(df, sg_empresa, nm_empresa, nr_voo, route, cd_tipo_linha,
                   sg_origem, tp_origem, nm_aerodromo_origem,
                   dt_partida_prevista, duracao_esperada, dt_partida_real, atraso_partida,
                   sg_destino, tp_destino, nm_aerodromo_destino,
                   dt_chegada_prevista, dt_chegada_real, duracao_real, atraso_chegada, tp_situacao,
                   cd_justificativa, ds_justificativa)

#--- kill old dataframe
rm("vra")

#--- rename the new one
vra <- df

#--- kill temporary dataframe
rm("df")

#--- generate destinations dataframe
print("Loading destinations.rda...")

load("destinations.rda")

print("destinations.rda uploaded successfully!")

#--- for the destinations
vra <- vra[order(vra$dt_chegada_real, vra$sg_destino),]

#--- merge VRA with destinations
#--- in order to fill the missing airports names
df<-merge(vra, destinations,
          by.x=c("sg_destino"),
          by.y=c("destination_icao"),
          all.x=TRUE,
          sort=TRUE)

#--- adjust the names 
df$tp_destino<-NULL
df$nm_aerodromo_destino<-NULL
names(df)[names(df) == "destination_type"] <- "tp_destino"
names(df)[names(df) == "destination_name"] <- "nm_aerodromo_destino"

#--- reorder fields
df = dplyr::select(df, sg_empresa, nm_empresa, nr_voo, route, cd_tipo_linha,
                   sg_origem, tp_origem, nm_aerodromo_origem,
                   dt_partida_prevista, duracao_esperada, dt_partida_real, atraso_partida,
                   sg_destino, tp_destino, nm_aerodromo_destino,
                   dt_chegada_prevista, dt_chegada_real, duracao_real, atraso_chegada, tp_situacao,
                   cd_justificativa, ds_justificativa)

#--- kill old dataframe
rm("vra")

#--- rename the new one
vra <- df

#--- kill temporary dataframe
rm("df")

#--- fill nm_aerodromo_origem not found
df<-merge(vra, airports,
          by.x=c("sg_destino"),
          by.y=c("airport_icao"),
          all.x=TRUE,
          sort=TRUE)

df$nm_aerodromo_destino <- ifelse(is.na(df$nm_aerodromo_destino),
                                 df$airport_name, df$nm_aerodromo_destino)

df$airport_name<-NULL

#--- reorder fields
df = dplyr::select(df, sg_empresa, nm_empresa, nr_voo, route, cd_tipo_linha,
                   sg_origem, tp_origem, nm_aerodromo_origem,
                   dt_partida_prevista, duracao_esperada, dt_partida_real, atraso_partida,
                   sg_destino, tp_destino, nm_aerodromo_destino,
                   dt_chegada_prevista, dt_chegada_real, duracao_real, atraso_chegada, tp_situacao,
                   cd_justificativa, ds_justificativa)

#--- kill old dataframe
rm("vra")

#--- rename the new one
vra <- df

#--- kill temporary dataframe
rm("df")

#--- adjust origin and destination types
vra <- transform(vra, tp_origem= ifelse(grepl('^SB', vra$sg_origem),"B", "O"))
vra <- transform(vra, tp_destino= ifelse(grepl('^SB', vra$sg_destino),"B", "O"))

#--- for the airlines
vra <- vra[order(vra$sg_empresa),]

#--- merge VRA with airlines in order to fill the missing airline names
df<-merge(vra, airlines,
          by.x=c("sg_empresa"),
          by.y=c("airline_icao"),
          all.x=TRUE,
          sort=TRUE)

#--- adjust the names
df$nm_empresa <- ifelse(is.na(df$nm_empresa),
                              df$airline_name, df$nm_empresa)
df$airline_name<-NULL

#--- reorder fields
df = dplyr::select(df, sg_empresa, nm_empresa, nr_voo, route, cd_tipo_linha,
                   sg_origem, tp_origem, nm_aerodromo_origem,
                   dt_partida_prevista, duracao_esperada, dt_partida_real, atraso_partida,
                   sg_destino, tp_destino, nm_aerodromo_destino,
                   dt_chegada_prevista, dt_chegada_real, duracao_real, atraso_chegada, tp_situacao,
                   cd_justificativa, ds_justificativa)

#--- kill old dataframe
rm("vra")

#--- rename the new one
vra <- df

#--- kill temporary dataframe
rm("df")

#--- for justifications
vra <- vra[order(vra$cd_justificativa),]

#--- merge VRA with justifications codes in order to fill the missing justifications
df<-merge(vra, justifications,
          by.x=c("cd_justificativa"),
          by.y=c("justification_code"),
          all.x=TRUE,
          sort=TRUE)

df$ds_justificativa <- ifelse(is.na(df$ds_justificativa),
                                   df$justification_description, df$ds_justificativa)

#--- adjust the names 
df$justification_description<-NULL

#--- reorder fields
df = dplyr::select(df, sg_empresa, nm_empresa, nr_voo, route, cd_tipo_linha,
                   sg_origem, tp_origem, nm_aerodromo_origem,
                   dt_partida_prevista, duracao_esperada, dt_partida_real, atraso_partida,
                   sg_destino, tp_destino, nm_aerodromo_destino,
                   dt_chegada_prevista, dt_chegada_real, duracao_real, atraso_chegada, tp_situacao,
                   cd_justificativa, ds_justificativa)

#--- kill old dataframe
rm("vra")

#--- rename the new one
vra <- df

#--- kill temporary dataframe
rm("df")

#--- create country colum for origins and destinations
vra$origin_country<-""
vra$destination_country<-""

vra$sg_origem_co<-substr(vra$sg_origem,1,2)
vra$sg_destino_co<-substr(vra$sg_destino,1,2)

vra <- vra[order(vra$sg_origem_co),]

df<-merge(vra, countries,
          by.x=c("sg_origem_co"),
          by.y=c("country_icao_code"),
          all.x=TRUE,
          sort=TRUE)

df$sg_origem_co<-NULL
df$origin_country<-NULL
names(df)[names(df) == "country_name"] <- "origin_country"

#--- rename the new one
vra <- df

#--- kill temporary dataframe
rm("df")

df<-merge(vra, countries,
          by.x=c("sg_destino_co"),
          by.y=c("country_icao_code"),
          all.x=TRUE,
          sort=TRUE)

df$sg_destino_co<-NULL
df$destination_country<-NULL
names(df)[names(df) == "country_name"] <- "destination_country"

#--- rename the new one
vra <- df

#--- kill temporary dataframe
rm("df")

#--- fill nm_aerodromo_origem and nm_aerodromo_destino == ""
vra <- vra[order(vra$sg_origem),]

df<-merge(vra, airports,
          by.x=c("sg_origem"),
          by.y=c("airport_icao"),
          all.x=TRUE,
          sort=TRUE)

df$nm_aerodromo_origem <- ifelse(df$nm_aerodromo_origem=="",
                              df$airport_name, df$nm_aerodromo_origem)

df$airport_name<-NULL

vra<-df

rm("df")

vra <- vra[order(vra$sg_destino),]

df<-merge(vra, airports,
          by.x=c("sg_destino"),
          by.y=c("airport_icao"),
          all.x=TRUE,
          sort=TRUE)

df$nm_aerodromo_destino <- ifelse(df$nm_aerodromo_destino=="",
                                 df$airport_name, df$nm_aerodromo_destino)

df$airport_name<-NULL

vra<-df

rm("df")

#--- Load ASOS
print("Loading asosBR...")

asosBR <- as.data.frame(read.table("asosBR.txt", header = TRUE,
                                   sep = ",", dec = ".", skip = 0,
                                   stringsAsFactors=FALSE))

print("asosBR successfully loaded!")

print("|Converting column temperature and dew point from fahrenheit to celsius...")

asosBR$tmpc <- as.numeric((as.numeric(asosBR$tmpf) - 32) * .5556)
asosBR$dwpc <- as.numeric((as.numeric(asosBR$dwpf) - 32) * .5556)

print("Column temperature successfully converted from fahrenheit to celsius!")

print("Loading the sequenceID, eventID and other weather characteristics columns...")

query<-"select replace(substr(valid,1,10),'-','') as sequenceID, "
query<-paste(query,"substr(valid,12,2) as eventID, ",sep=" ")
query<-paste(query,"station as airport, relh as humidity, ",sep=" ")
query<-paste(query,"drct as wind_direction, ",sep=" ")
query<-paste(query,"sknt as wind_speed, mslp as pressure, ",sep=" ")
query<-paste(query,"vsby as visibility, skyc1 as cloudiness, ",sep=" ")
query<-paste(query,"skyl1 as ceiling, tmpc as temperature, ",sep=" ")
query<-paste(query,"dwpc as dew_point ",sep=" ")
query<-paste(query,"from asosBR ",sep=" ")
query<-paste(query,"group by sequenceID, eventID, airport ",sep=" ")
asosBR<-sqldf(query)

print("SequenceID, eventID and other weather characteristics loaded successfully!")

print("Sorting and grouping weather data...")

asosBR <- asosBR[order(asosBR$sequenceID,asosBR$eventID,asosBR$airport),]

print("Weather data sorted and grouped successfully!")

print("vra datetime transformations necessary for integration with asos...")

# create 2 lists with all values from expected departure date (index column=10)
expected_depart_d<-sapply(strsplit(vra[,10], " ", fixed = TRUE), "[", 1)
expected_depart_h<-sapply(strsplit(vra[,10], " ", fixed = TRUE), "[", 2)

# create 2 lists with all values from real departure date (index column=12)
real_depart_d<-sapply(strsplit(vra[,12], " ", fixed = TRUE), "[", 1)
real_depart_h<-sapply(strsplit(vra[,12], " ", fixed = TRUE), "[", 2)

# create 2 lists with all values from expected arrival date (index column=16)
expected_arrival_d<-sapply(strsplit(vra[,16], " ", fixed = TRUE), "[", 1)
expected_arrival_h<-sapply(strsplit(vra[,16], " ", fixed = TRUE), "[", 2)

# create 2 lists with all values from real arrival date (index column=17)
real_arrival_d<-sapply(strsplit(vra[,17], " ", fixed = TRUE), "[", 1)
real_arrival_h<-sapply(strsplit(vra[,17], " ", fixed = TRUE), "[", 2)

# create a temporary dataframe with these 8 columns
intermed_df <- data.frame("expected_depart_date" = expected_depart_d,
                          "expected_depart_time" = expected_depart_h,
                          "real_depart_date" = real_depart_d,
                          "real_depart_time" = real_depart_h,
                          "expected_arrival_date" = expected_arrival_d,
                          "expected_arrival_time" = expected_arrival_h,
                          "real_arrival_date" = real_arrival_d,
                          "real_arrival_time" = real_arrival_h)

# remove original datetime columns from vra
vra$dt_partida_prevista<-NULL
vra$dt_chegada_prevista<-NULL
vra$dt_partida_real<-NULL
vra$dt_chegada_real<-NULL

# binds intermed_df with vra
vra <- cbind(vra,intermed_df)

# transform vra date and time to character in order to integrate with asos
vra$expected_depart_date<-as.character(gsub("-","",as.character(vra$expected_depart_date)))
vra$expected_depart_hour<-substr(as.character(gsub(":","",as.character(vra$expected_depart_time))),1,2)
vra$expected_depart_time<-NULL

vra$real_depart_date<-as.character(gsub("-","",as.character(vra$real_depart_date)))
vra$real_depart_hour<-substr(as.character(gsub(":","",as.character(vra$real_depart_time))),1,2)
vra$real_depart_time<-NULL

vra$expected_arrival_date<-as.character(gsub("-","",as.character(vra$expected_arrival_date)))
vra$expected_arrival_hour<-substr(as.character(gsub(":","",as.character(vra$expected_arrival_time))),1,2)
vra$expected_arrival_time<-NULL

vra$real_arrival_date<-as.character(gsub("-","",as.character(vra$real_arrival_date)))
vra$real_arrival_hour<-substr(as.character(gsub(":","",as.character(vra$real_arrival_time))),1,2)
vra$real_arrival_time<-NULL

# remove temporary dataframe and lists
rm("intermed_df")
rm("expected_arrival_d")
rm("expected_arrival_h")
rm("expected_depart_d")
rm("expected_depart_h")
rm("real_arrival_d")
rm("real_arrival_h")
rm("real_depart_d")
rm("real_depart_h")

print("Loading weather conditions for flight departures...")

#--- Step 1: It brings the climatic conditions to the
# --- origin airports, ie departures
vra <- vra[order(vra$real_depart_date,vra$real_depart_hour,vra$sg_origem),]

partidas<-merge(vra, asosBR,
                by.x=c("real_depart_date",
                       "real_depart_hour",
                       "sg_origem"),
                by.y=c("sequenceID",
                       "eventID",
                       "airport"),
                all.x=TRUE,
                sort=TRUE)

partidas$depart_humidity<-NULL
partidas$depart_wind_speed<-NULL
partidas$depart_pressure<-NULL
partidas$depart_visibility<-NULL
partidas$depart_cloudiness<-NULL
partidas$depart_ceiling<-NULL
partidas$depart_temperature<-NULL
partidas$depart_dew_point<-NULL
partidas$depart_humidity<-NULL

partidas$arrival_humidity<-NULL
partidas$arrival_wind_speed<-NULL
partidas$arrival_pressure<-NULL
partidas$arrival_visibility<-NULL
partidas$arrival_cloudiness<-NULL
partidas$arrival_ceiling<-NULL
partidas$arrival_temperature<-NULL
partidas$arrival_dew_point<-NULL
partidas$arrival_humidity<-NULL

colnames(partidas)[colnames(partidas)=="humidity"] <- "depart_humidity"
colnames(partidas)[colnames(partidas)=="wind_speed"] <- "depart_wind_speed"
colnames(partidas)[colnames(partidas)=="pressure"] <- "depart_pressure"
colnames(partidas)[colnames(partidas)=="visibility"] <- "depart_visibility"
colnames(partidas)[colnames(partidas)=="cloudiness"] <- "depart_cloudiness"
colnames(partidas)[colnames(partidas)=="ceiling"] <- "depart_ceiling"
colnames(partidas)[colnames(partidas)=="temperature"] <- "depart_temperature"
colnames(partidas)[colnames(partidas)=="dew_point"] <- "depart_dew_point"
colnames(partidas)[colnames(partidas)=="wind_direction"] <- "depart_wind_direction"

query<-"select * from partidas order by real_depart_date, real_depart_hour, sg_destino"
partidas<-sqldf(query)

rm("vra")

print("Weather conditions for flight departures successfully loaded!")

print("Loading weather conditions for flight arrivals...")

#--- Step 2: Bring the weather conditions to the
# --- destination airports, ie arrivals
chegadas<-merge(partidas, asosBR,
                by.x=c("real_depart_date",
                       "real_depart_hour",
                       "sg_destino"),
                by.y=c("sequenceID",
                       "eventID",
                       "airport"),
                all.x=TRUE,
                sort=TRUE)

colnames(chegadas)[colnames(chegadas)=="humidity"] <- "arrival_humidity"
colnames(chegadas)[colnames(chegadas)=="wind_speed"] <- "arrival_wind_speed"
colnames(chegadas)[colnames(chegadas)=="pressure"] <- "arrival_pressure"
colnames(chegadas)[colnames(chegadas)=="visibility"] <- "arrival_visibility"
colnames(chegadas)[colnames(chegadas)=="cloudiness"] <- "arrival_cloudiness"
colnames(chegadas)[colnames(chegadas)=="ceiling"] <- "arrival_ceiling"
colnames(chegadas)[colnames(chegadas)=="temperature"] <- "arrival_temperature"
colnames(chegadas)[colnames(chegadas)=="dew_point"] <- "arrival_dew_point"
colnames(chegadas)[colnames(chegadas)=="wind_direction"] <- "arrival_wind_direction"

rm("partidas")

vra_asosBR<-chegadas

rm("chegadas")

vra_asosBR$depart_humidity<-as.numeric(vra_asosBR$depart_humidity)
vra_asosBR$depart_wind_speed<-as.numeric(vra_asosBR$depart_wind_speed)
vra_asosBR$depart_dew_point<-as.numeric(vra_asosBR$depart_dew_point)
vra_asosBR$depart_wind_direction<-as.numeric(vra_asosBR$depart_wind_direction)
vra_asosBR$arrival_humidity<-as.numeric(vra_asosBR$arrival_humidity)
vra_asosBR$arrival_wind_speed<-as.numeric(vra_asosBR$arrival_wind_speed)
vra_asosBR$arrival_dew_point<-as.numeric(vra_asosBR$arrival_dew_point)
vra_asosBR$arrival_wind_direction<-as.numeric(vra_asosBR$arrival_wind_direction)

print("Weather conditions for flight arrivals successfully loaded!")

# fill with mean values for NA values
vra_asosBR$depart_temperature[is.na(vra_asosBR$depart_temperature)] <- 24 # average temperature
vra_asosBR$arrival_temperature[is.na(vra_asosBR$arrival_temperature)] <- 22 # average temperature
vra_asosBR$depart_dew_point[is.na(vra_asosBR$depart_dew_point)] <- 22 # average Dew point
vra_asosBR$arrival_dew_point[is.na(vra_asosBR$arrival_dew_point)] <- 22 # average Dew point
vra_asosBR$depart_humidity[is.na(vra_asosBR$depart_humidity)] <- 70 # average Relative humidity
vra_asosBR$arrival_humidity[is.na(vra_asosBR$arrival_humidity)] <- 70 # average Relative humidity
vra_asosBR$depart_pressure[is.na(vra_asosBR$depart_pressure)] <- 1015 # average Pressure
vra_asosBR$arrival_pressure[is.na(vra_asosBR$arrival_pressure)] <- 1015 # average Pressure
vra_asosBR$depart_wind_speed[is.na(vra_asosBR$depart_wind_speed)] <- 6 # average speed
vra_asosBR$arrival_wind_speed[is.na(vra_asosBR$arrival_wind_speed)] <- 6 # average speed

#---------------- Discretization

# depart and arrival wind speed
# Beaufort Scale -- Ref.: https://www.spc.noaa.gov/faq/tornado/beaufort.html
# Method: Concept Hierarchy

# --------- On Land
# "Calm" Calm, smoke rises vertically
# "Light Air" Smoke drift indicates wind direction, still wind vanes
# "Light Breeze" Wind felt on face, leaves rustle, vanes begin to move
# "Gentle Breeze" Leaves and small twigs constantly moving, light flags extended
# "Moderate Breeze" Dust, leaves, and loose paper lifted, small tree branches move
# "Fresh Breeze" Small trees in leaf begin to sway
# "Strong Breeze" Larger tree branches moving, whistling in wires
# "Near Gale" Whole trees moving, resistance felt walking against wind
# "Gale" Twigs breaking off trees, generally impedes progress
# "Strong Gale" Slight structural damage occurs, slate blows off roofs
# "Storm" Seldom experienced on land, trees broken or uprooted, "considerable structural damage"
# "Violent Storm"
# "Hurricane"

vra_asosBR$"ds_depart_wind_speed" <- ordered(cut(as.numeric(vra_asosBR$"depart_wind_speed"),
                                           c(0,1,3,6,10,16,21,27,33,40,47,55,63,100)),
                                       labels = c("Calm",
                                                 "Light Air",
                                                 "Light Breeze",
                                                 "Gentle Breeze",
                                                 "Moderate Breeze",
                                                 "Fresh Breeze",
                                                 "Strong Breeze",
                                                 "Near Gale",
                                                 "Gale",
                                                 "Strong Gale",
                                                 "Storm",
                                                 "Violent Storm",
                                                 "Hurricane"))

vra_asosBR$ds_depart_wind_speed<-as.character(vra_asosBR$ds_depart_wind_speed)
vra_asosBR$ds_depart_wind_speed[is.na(vra_asosBR$ds_depart_wind_speed)] = "Not Informed"
vra_asosBR$ds_depart_wind_speed<-as.factor(vra_asosBR$ds_depart_wind_speed)

vra_asosBR$"ds_arrival_wind_speed" <- ordered(cut(as.numeric(vra_asosBR$"arrival_wind_speed"),
                                           c(0,1,3,6,10,16,21,27,33,40,47,55,63,100)),
                                       labels = c("Calm",
                                                  "Light Air",
                                                  "Light Breeze",
                                                  "Gentle Breeze",
                                                  "Moderate Breeze",
                                                  "Fresh Breeze",
                                                  "Strong Breeze",
                                                  "Near Gale",
                                                  "Gale",
                                                  "Strong Gale",
                                                  "Storm",
                                                  "Violent Storm",
                                                  "Hurricane"))

vra_asosBR$ds_arrival_wind_speed<-as.character(vra_asosBR$ds_arrival_wind_speed)
vra_asosBR$ds_arrival_wind_speed[is.na(vra_asosBR$ds_arrival_wind_speed)] = "Not Informed"
vra_asosBR$ds_arrival_wind_speed<-as.factor(vra_asosBR$ds_arrival_wind_speed)

# depart and arrival wind direction
vra_asosBR$"ds_depart_wind_direction" <- ordered(cut(as.numeric(vra_asosBR$"depart_wind_direction"),
                                           c(0,11,33,56,78,101,123,146,168,191,213,236,258,281,303,326,348,360)),
                                       labels = c("N",
                                                  "NNE",
                                                  "NE",
                                                  "ENE",
                                                  "E",
                                                  "ESE",
                                                  "SE",
                                                  "SSE",
                                                  "S",
                                                  "SSW",
                                                  "SW",
                                                  "WSW",
                                                  "W",
                                                  "WNW",
                                                  "NW",
                                                  "NNW",
                                                  "N"))

vra_asosBR$ds_depart_wind_direction<-as.character(vra_asosBR$ds_depart_wind_direction)
vra_asosBR$ds_depart_wind_direction[is.na(vra_asosBR$ds_depart_wind_direction)] = "Not Informed"
vra_asosBR$ds_depart_wind_direction<-as.factor(vra_asosBR$ds_depart_wind_direction)

vra_asosBR$"ds_arrival_wind_direction" <- ordered(cut(as.numeric(vra_asosBR$"arrival_wind_direction"),
                                                     c(0,11,33,56,78,101,123,146,168,191,213,236,258,281,303,326,348,360)),
                                                 labels = c("N",
                                                            "NNE",
                                                            "NE",
                                                            "ENE",
                                                            "E",
                                                            "ESE",
                                                            "SE",
                                                            "SSE",
                                                            "S",
                                                            "SSW",
                                                            "SW",
                                                            "WSW",
                                                            "W",
                                                            "WNW",
                                                            "NW",
                                                            "NNW",
                                                            "N"))

vra_asosBR$ds_arrival_wind_direction<-as.character(vra_asosBR$ds_arrival_wind_direction)
vra_asosBR$ds_arrival_wind_direction[is.na(vra_asosBR$ds_arrival_wind_direction)] = "Not Informed"
vra_asosBR$ds_arrival_wind_direction<-as.factor(vra_asosBR$ds_arrival_wind_direction)

# time of the day
vra_asosBR$"ds_depart_day_period" <- ordered(cut(as.numeric(vra_asosBR$"real_depart_hour"),
                                          c(-1,4,8,10,12,16,19,22,24)),
                                      labels = c("Night",
                                                 "Early Morning",
                                                 "Mid Morning",
                                                 "Late Morning",
                                                 "Afternoon",
                                                 "Early Evening",
                                                 "Late Evening", "Night"))

vra_asosBR$"ds_arrival_day_period" <- ordered(cut(as.numeric(vra_asosBR$"real_arrival_hour"),
                                                  c(-1,4,8,10,12,16,19,22,24)),
                                              labels = c("Night",
                                                         "Early Morning",
                                                         "Mid Morning",
                                                         "Late Morning",
                                                         "Afternoon",
                                                         "Early Evening",
                                                         "Late Evening", "Night"))

#--- Reordering columns
vra_asosBR = dplyr::select(vra_asosBR, nr_voo, sg_empresa, nm_empresa, cd_tipo_linha,
                           sg_origem, nm_aerodromo_origem, origin_country,
                           expected_depart_date, expected_depart_hour, real_depart_date,
                           real_depart_hour,
                           duracao_esperada, atraso_partida,
                           depart_temperature, 
                           depart_dew_point, 
                           depart_humidity, 
                           depart_pressure, 
                           depart_visibility, 
                           depart_cloudiness, depart_ceiling,
                           depart_wind_speed, ds_depart_wind_speed,
                           depart_wind_direction, ds_depart_wind_direction,
                           ds_depart_day_period,
                           sg_destino, nm_aerodromo_destino, destination_country,
                           expected_arrival_date, expected_arrival_hour,
                           real_arrival_date, real_arrival_hour,
                           duracao_real, atraso_chegada,
                           arrival_temperature, 
                           arrival_dew_point, 
                           arrival_humidity, 
                           arrival_pressure, 
                           arrival_visibility, 
                           arrival_cloudiness, arrival_ceiling,
                           arrival_wind_speed, ds_arrival_wind_speed,
                           arrival_wind_direction, ds_arrival_wind_direction,
                           ds_arrival_day_period,
                           tp_situacao, cd_justificativa, ds_justificativa)

#--- Renaming columns
names(vra_asosBR)[names(vra_asosBR) == "nr_voo"] <- "flight_id"
names(vra_asosBR)[names(vra_asosBR) == "sg_empresa"] <- "airline_icao"
names(vra_asosBR)[names(vra_asosBR) == "nm_empresa"] <- "airline_name"
names(vra_asosBR)[names(vra_asosBR) == "cd_tipo_linha"] <- "linetype_code"
names(vra_asosBR)[names(vra_asosBR) == "sg_origem"] <- "origin_icao"

names(vra_asosBR)[names(vra_asosBR) == "nm_aerodromo_origem"] <- "origin_name"
names(vra_asosBR)[names(vra_asosBR) == "duracao_esperada"] <- "expected_duration"
names(vra_asosBR)[names(vra_asosBR) == "atraso_partida"] <- "departaure_delay"

names(vra_asosBR)[names(vra_asosBR) == "sg_destino"] <- "destination_icao"

names(vra_asosBR)[names(vra_asosBR) == "nm_aerodromo_destino"] <- "destination_name"
names(vra_asosBR)[names(vra_asosBR) == "duracao_real"] <- "real_duration"
names(vra_asosBR)[names(vra_asosBR) == "atraso_chegada"] <- "arrival_delay"

names(vra_asosBR)[names(vra_asosBR) == "tp_situacao"] <- "situation_type"
names(vra_asosBR)[names(vra_asosBR) == "cd_justificativa"] <- "justification_code"
names(vra_asosBR)[names(vra_asosBR) == "ds_justificativa"] <- "justification_description"

print("Saving Brazilian Flights Dataset...")

bfd<-vra_asosBR

rm("vra_asosBR")

#--- categorical record as a factor
bfd$flight_id<-as.factor(bfd$flight_id)
bfd$airline_icao<-as.factor(bfd$airline_icao)
bfd$airline_name<-as.factor(bfd$airline_name)
bfd$linetype_code<-as.factor(bfd$linetype_code)
bfd$origin_icao<-as.factor(bfd$origin_icao)

bfd$origin_name<-as.factor(bfd$origin_name)
bfd$origin_country<-as.factor(bfd$origin_country)
bfd$expected_depart_date<-as.factor(bfd$expected_depart_date)
bfd$expected_depart_hour<-as.factor(bfd$expected_depart_hour)
bfd$real_depart_date<-as.factor(bfd$real_depart_date)
bfd$real_depart_hour<-as.factor(bfd$real_depart_hour)

bfd$depart_visibility<-as.numeric(bfd$depart_visibility)
bfd$depart_cloudiness<-as.numeric(bfd$depart_cloudiness)
bfd$depart_ceiling<-as.numeric(bfd$depart_ceiling)

bfd$destination_icao<-as.factor(bfd$destination_icao)

bfd$destination_name<-as.factor(bfd$destination_name)
bfd$destination_country<-as.factor(bfd$destination_country)
bfd$expected_arrival_date<-as.factor(bfd$expected_arrival_date)
bfd$expected_arrival_hour<-as.factor(bfd$expected_arrival_hour)
bfd$real_arrival_date<-as.factor(bfd$real_arrival_date)
bfd$real_arrival_hour<-as.factor(bfd$real_arrival_hour)

bfd$arrival_visibility<-as.numeric(bfd$arrival_visibility)
bfd$arrival_cloudiness<-as.numeric(bfd$arrival_cloudiness)
bfd$arrival_ceiling<-as.numeric(bfd$arrival_ceiling)

bfd$situation_type<-as.factor(bfd$situation_type)
bfd$justification_code<-as.factor(bfd$justification_code)
bfd$justification_description<-as.factor(bfd$justification_description)

bfd$ds_depart_wind_speed<-as.ordered(bfd$ds_depart_wind_speed)
bfd$ds_arrival_wind_speed<-as.ordered(bfd$ds_arrival_wind_speed)

#--- delete 7 flights which real depart date and arrival date are till 2019
bfd <- bfd %>% filter (as.numeric(substr(as.character(real_depart_date),1,4)) <= 2019)
bfd <- bfd %>% filter (as.numeric(substr(as.character(real_arrival_date),1,4)) <= 2019)

#--- adjust the name of departure delay column
colnames(bfd)[colnames(bfd) == "departaure_delay"] <- "departure_delay"

#--- remove cloudiness
bfd$depart_cloudiness<-NULL
bfd$arrival_cloudiness<-NULL

#--- remove ceiling
bfd$depart_ceiling<-NULL
bfd$arrival_ceiling<-NULL

print("Saving Brazilian Flights Dataset (BFD)...")
save(bfd, file="bfd.rda")

print("bfd.rda saved successfully!")

print(paste("Number of tuples in the original dataset (vra):",originalVRAtuples))
print(paste("Number of tuples in the integrated dataset (bfd):",nrow(bfd)))

print("Cleaning the working environment ...")

rm("query")

print("Working environment ready!")

[1] "Loading work packages..."


network: Classes for Relational Data
Version 1.16.0 created on 2019-11-30.
copyright (c) 2005, Carter T. Butts, University of California-Irvine
                    Mark S. Handcock, University of California -- Los Angeles
                    David R. Hunter, Penn State University
                    Martina Morris, University of Washington
                    Skye Bender-deMoll, University of Washington
 For citation information, type citation("network").
 Type help("network-package") to get started.

------------------------------------------------------------------------------
You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)
------------------------------------------------------------------------------

Attaching package: ‘plyr’

The following object is masked from ‘package:network’:

    is.discrete

The following objects are masked from ‘package:dply

[1] "Work packages loaded successfully!"
[1] "Loading vra_2000_2018.RData..."
[1] "vra_2000_2018.RData uploaded successfully!"
[1] "Loading asosBR..."


Warning message in scan(file = file, what = what, sep = sep, quote = quote, dec = dec, :
“EOF within quoted string”

[1] "asosBR successfully loaded!"
[1] "|Converting column temperature and dew point from fahrenheit to celsius..."


...

[1] "vra successfully loaded and filtered!"
[1] "Loading weather conditions for flight departures..."
[1] "Weather conditions for successfully loaded flight departures!"
[1] "Loading weather conditions for flight arrivals..."


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

[1] "Datasets successfully unified!"


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

[1] "Recording vra_asosBR.Rda dataset..."
[1] "vra_asosBR.Rda saved successfully!"
[1] "Cleaning the working environment ..."
[1] "Working environment ready!"


**4. Result**

The final result of the work consists of a single dataset called ** bfd.rda ** with 15,505,922 observations or tuples and 45 characteristics or variables, described below:

1. **flight_id** - ANAC flight identifier;<br>
2. **airline_icao** - ICAO acronym of the airline. Ex: GLO, TAM, ONE;<br>
3. **airline_name** - Full name of the airline;<br>
4. **lynetype_code** - (N) National, (I) Intenational, (R) Regional, (H) Sub-regional, (E) Special, (C) Freight / Cargo, (G) International Freight / Cargo and (L) Postal Network;<br>
5. **origin_icao** - ICAO code of the flight origin airport;<br>
6. **origin_country** - ISO Country Name where the origin airport is located;<br>
7. **origin_name** - Full name of the airport of origin of the flight;<br>
8. **expected_depart_date** - Expected flight departure date in YYYYMMDD format where YYYY = year, MM = month and DD = day;<br>
9. **expected_depart_hour** - Estimated flight departure hour in HH format where HH = full hour;<br>
10. **real_depart_date** - Flight departure date in YYYYMMDD format where YYYY = year, MM = month and DD = day;<br>
11. **real_depart_hour** - Flight departure in HH format where HH = full hour;<br>
12. **expected_duration** - Difference in minutes between expected departure and arrival datetime;<br**>
13. **departure_delay** - Difference in minutes between expected and real departure datetime;<br>
14. **depart_temperature** - Temperature in degrees Celsius at the airport of origin, at the time of flight departure;<br>
15. **depart_dew_point** - Dew point, in degrees Celsius at the airport of origin, at the time of flight departure;<br>
16. **depart_humidity** - Percentage of relative humidity in the airport of origin;<br>
17. **depart_pressure** - Atmospheric pressure, in mbar, at the airport of origin;<br>
18. **depart_visibility** - Visibility, in miles, at the airport of origin;<br>
19. **depart_wind_speed** - Wind speed, in knots, at the airport of origin;<br>
20. **depart_wind_direction** - Wind direction, based on Wind Rose, at the airport of origin;<br>
21. **destination_icao** - ICAO code of the destination airport of the flight;<br>
22. **destination_name** - Full name of the airport of destination of the flight;<br>
23. **destination_country** - ISO country name where the destination airport is located;<br>
24. **expected_arrival_date** - Estimated date and time of flight arrival in YYYYMMDD format where YYYY = year, MM = month and DD = day;<br>
25. **expected_arrival_hour** - Estimated flight arrival hour in HH format where HH = full hour;<br>
26. **real_arrival_date** - Flight arrival date in YYYYMMDD format where YYYY = year, MM = month and DD = day;<br>
27. **real_arrival_hour** - Flight arrival hour in HH format where HH = full hour;<br>
28. **real_duration** - Difference in minutes between real departure and arrival datetime;<br>
29. **arrival_delay** - Difference in minutes between expected and real arrival datetime;<br>
30. **arrival_temperature** - Temperature in degrees Celsius at the destination airport, at the time of flight arrival;<br>
31. **arrival_dew_point** - Dew point, in degrees Celsius at the destination airport, at the time of flight arrival.<br>
32. **arrival_humidity** - Percentage of relative humidity in the destination airport;<br>
33. **arrival_pressure** - Atmospheric pressure, in mbar, at the destination airport;<br>
34. **arrival_visibility** - Visibility, in miles, at the destination airport;<br>
35. **arrival_wind_speed** - Wind speed in knots at the destination airport;<br>
36. **arrival_wind_direction** - Wind direction, based on Wind Rose, at the airport of destination;<br>
37. **situation_type** - Flight status;<br>
38. **justification_code** - Flight delay or cancellation justification code;<br>
39. **justification_description** - Description of justification for flight delay or cancellation;<br><br>
**Discretized fields (ds_):**<br>
40. **ds_depart_wind_speed** - The wind classification based on Belfourt Scale at the origin airport;<br>
41. **ds_arrival_wind_speed** - The wind classification based on Belfourt Scale at the destination airport;<br>
42. **ds_depart_wind_direction** - The acronymn for the wind direction at destination airport;<br>
43. **ds_arrival_wind_direction** - The acronymn for the wind direction at destination airport;<br>
44. **ds_depart_day_period** - Period of the day on real departure; and<br>
45. **ds_arrival_day_period** - Period of the day on real arrival.

**References**

[WUA] End of service for the weather underground api. https://
apicommunity.wunderground.com/weatherapi/topics/
end-of-service-for-the-weather-underground-api. Accessed:
2020-02-20.<br>
[MET] Metar help. https://weather.cod.edu/notes/metar.html. Accessed:
2020-02-20.<br>
[IAC] Procedimentos para o registro de alteração em voos de empresas de transporte
aéreo regular. http://www.anac.gov.br/assuntos/legislacao/
legislacao-1/iac-e-is/iac/iac-1504. Accessed: 2020-02-20.<br>
[ASO] Us national weather service - automated surface observing systems. https://
www.weather.gov/asos/. Accessed: 2020-07-24.